In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unrar x /content/drive/MyDrive/anm_casme_sq.rar

Streaming output truncated to the last 5000 lines.
Extracting  casme_sq/happy/4579.jpg                                       47%  OK 
Extracting  casme_sq/happy/4580.jpg                                       47%  OK 
Extracting  casme_sq/happy/4581.jpg                                       47%  OK 
Extracting  casme_sq/happy/4582.jpg                                       47%  OK 
Extracting  casme_sq/happy/4583.jpg                                       47%  OK 
Extracting  casme_sq/happy/4584.jpg                                       47%  OK 
Extracting  casme_sq/happy/4585.jpg                                       47%  OK 
Extracting  casme_sq/happy/4586.jpg                                       47%  OK 
Extracting  casme_sq/happy/4587.jpg                                       47%  OK 
Extracting  casme_sq/happy/4588.jpg                                       47%  OK 
Extracting  casme_sq/happy/45

In [3]:
import numpy as np
import pandas as pd 
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from skimage import data, color, feature
from keras.utils.vis_utils import plot_model
import os
from tqdm import tqdm
from sklearn import preprocessing
from keras.applications import MobileNet
from sklearn.model_selection import train_test_split
import cv2

Zdf_train = pd.read_csv('/content/drive/MyDrive/anm_casme_sq_labled.csv') # Fall No fall
Zdf_train.head()

Ztargets_series = pd.Series(Zdf_train['class'])

Zone_hot = pd.get_dummies(Ztargets_series, sparse = True)

Zone_hot_labels = np.asarray(Zone_hot)

Zim_size1 = 128
Zim_size2 = 128

Zx_train = []
Zy_train = []
Zx_test = []

i = 0 
#for folder in range(7):
# total pictures are 17124 and i test on 10000

for folder, f in tqdm(Zdf_train.values[:17124]): 
  if type(cv2.imread('/content/casme_sq/{}/{}'.format(folder,f)))==type(None): 
    continue 
  else: 
    img = cv2.imread('/content/casme_sq/{}/{}'.format(folder,f)) 
    label = Zone_hot_labels[i] 
    Zx_train.append(cv2.resize(img, (Zim_size1, Zim_size2))) 
    Zy_train.append(label) 
  i += 1 
np.save('Zx_train2',Zx_train) 
np.save('Zy_train2',Zy_train) 
print('Resize Done')

Zx_train= np .load('Zx_train2.npy')
Zy_tran = np.load('Zy_train2.npy')

Zy_train_raw = np.array(Zy_train, np.uint8)
Zx_train_raw = np.array(Zx_train, np.float32)

print(Zx_train_raw.shape)
print(Zy_train_raw.shape)

ZX_train, ZX_valid, ZY_train, ZY_valid = train_test_split(Zx_train_raw, Zy_train_raw, test_size=0.1, random_state=1)

num_class = Zy_train_raw.shape[1]

100%|██████████| 9574/9574 [00:12<00:00, 748.24it/s]


Resize Done
(9574, 128, 128, 3)
(9574, 8)


In [4]:

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend 
from keras.applications import ResNet50
base_model = ResNet50(weights = None, include_top=False, input_shape=(Zim_size1, Zim_size2, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.1)(x)
predictions = Dense(num_class, activation='softmax')(x)
# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy',f1_m,precision_m, recall_m])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

model.fit(ZX_train, ZY_train, epochs=10, workers=10,
    use_multiprocessing = True,
    validation_data=(ZX_valid, ZY_valid), verbose=1)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________